# Perform $H$-$\kappa$ stacking on seismic network deployment

In [ ]:
import os
# from collections import defaultdict
# import time
import pickle as pkl

import numpy as np
import rf
import rf.imaging
import matplotlib.pyplot as plt
import scipy
from scipy import signal

import obspy
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

## Read source file

In [ ]:
# rf_type = 'ZRT_td'
rf_type = 'ZRT_it'
# rf_type = 'LQT_td'
# rf_type = 'ZRT_fd'
# rf_type = 'LQT_fd'

In [ ]:
network = 'OA'
# network = '7X'
target_station = 'BT23'
# target_station = 'CE26'

In [ ]:
src_file = r"..\DATA\OA_rfs_20170911T000036-20181128T230620_{}_rev10_qual.h5".format(rf_type)
# src_file = r"..\DATA\OA_rfs_20170911T000036-20181128T230620_{}_rev9_qual.h5".format(rf_type)
# src_file = r"..\DATA\7X_rfs_20090616T034200-20110401T231849_{}_rev1_qual.h5".format(rf_type)

# data_all = rf_util.read_h5_rf(src_file)
data_all = rf_util.read_h5_rf(src_file, network, target_station, loc='0M')
# data_all = rf_util.read_h5_rf(src_file, network, target_station)

In [ ]:
type(data_all)

In [ ]:
# Visualize approximate direction distribution of incoming rays
# ppts = data_all.select(station=target_station).ppoints(50.0)
# fig = rf.imaging.plot_ppoints(ppts, color='C0')
# plt.title("{} source directions".format(target_station))
# plt.show()

In [ ]:
# # See if default DBSCAN grouping is effective collecting RFs from similar back_azimuths. Only show those allocated to a group
# grouping = np.array([tr.stats.get('rf_group') for tr in data_all.select(station=target_station)])
# grouping_mask = (grouping != None)
# fig = rf.imaging.plot_ppoints(ppts[grouping_mask, :], c=['C{}'.format(g) for g in grouping[grouping_mask]], alpha=0.5)
# plt.title("{} source directions".format(target_station))
# plt.show()

## Convert RFStream to dict database for convenient iteration and addressing

In [ ]:
db = rf_util.rf_to_dict(data_all)

## Select test station and channel

In [ ]:
station_db = db[target_station]

In [ ]:
channel = rf_util.choose_rf_source_channel(rf_type, station_db)
print("Selected channel: {}".format(channel))
channel_data = station_db[channel]
len(channel_data)

In [ ]:
# Check if there are any traces with NaNs in them. RF quality filtering prior to this SHOULD have removed any such traces.
np.sum([np.any(np.isnan(tr.data)) for tr in channel_data])

## Examine available metadata in each trace

In [ ]:
# channel_data[0].stats

## Apply quality filter to traces

In [ ]:
rf_util.label_rf_quality_simple_amplitude(rf_type, channel_data)
rf_stream_A = rf.RFStream([tr for tr in channel_data if tr.stats.predicted_quality == 'a'])
len(rf_stream_A)

## Plot RFs for traces according to quality

### Narrow data to events of a certain minimum magnitude and teleseismic distance

In [ ]:
# min_mag = 5.5
# max_mag = 7.0
# min_dist = 30
# max_dist = 90
# rf_stream_A = rf.RFStream([tr for tr in rf_stream_A if min_mag <= tr.stats.event_magnitude <= max_mag and
#                            min_dist <= tr.stats.distance <= max_dist]).sort(['back_azimuth'])
len(rf_stream_A)

### Predicted Quality A

In [ ]:
rf_data = [tr for tr in rf_stream_A]
rf_data = sorted(rf_data, key=lambda v: v.stats.back_azimuth)
_ = rf_plot_utils.plot_rf_stack(rf.RFStream(rf_data), trace_height=0.2, time_window=(-10, 30))

### Predicted Quality B

In [ ]:
rf_data = [tr for tr in channel_data if tr.stats.predicted_quality == 'b']
rf_data = sorted(rf_data, key=lambda v: v.stats.back_azimuth)
rf_plot_utils.plot_rf_stack(rf.RFStream(rf_data), trace_height=0.2, time_window=(-10, 30))
rf_bad = rf_data.copy()

***

# Plot HK stacks

In [ ]:
hk_data = {channel: [tr for tr in rf_stream_A]}
hk_all = {channel: [tr for tr in channel_data]}
hk_bad = {channel: [tr for tr in rf_bad]}

In [ ]:
def produce_hk_stacking(station_data, channel, V_p=6.4, weighting=(0.5, 0.5, 0.0), plot_layers=False, save_file=None):

    k_grid, h_grid, hk_stack = rf_stacking.compute_hk_stack(station_data, channel, h_range=np.linspace(20.0, 70.0, 501), root_order=2, V_p=V_p)

    # Normalize the stacked amplitudes of each phase before computing weighted sum, to ensure the
    # weights are meaningful in an absolute sense. Otherwise the weightings are relative to the mean
    # amplitude of the return of a given phase, which is somewhat arbitrary.
#     for i in range(3):
#         print(np.max(hk_stack[i, :, :]))
#         hk_stack[i, :, :] = hk_stack[i, :, :]/np.max(hk_stack[i, :, :])
    for i in range(3):
        print(np.max(hk_stack[i, :, :]))

    # Sum the phases
    hk_stack_sum = rf_stacking.compute_weighted_stack(hk_stack, weighting)

    # Raise the final sum over phases to power >1 to increase contrast
    hk_stack_sum = rf_util.signed_nth_power(hk_stack_sum, 2)
    hk_stack_sum = hk_stack_sum/np.nanmax(hk_stack_sum[:])
    
    sta = station_data[channel][0].stats.station
    num = len(station_data[channel])
    rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack_sum, title=sta + '.{}'.format(channel), num=num, save_file=save_file)
    if plot_layers:
        if save_file is not None:
            assert False, "NYI"
        else:
            rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack[0], title=sta + '.{} Ps'.format(channel), num=num, clip_negative=False)
            rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack[1], title=sta + '.{} PpPs'.format(channel), num=num, clip_negative=False)
            rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack[2], title=sta + '.{} PpSs + PsPs'.format(channel), num=num, clip_negative=False)
    # end if
# end func

In [ ]:
len(hk_data[channel])

In [ ]:
# Plot stack
weighting = (0.35, 0.35, 0.3)
# weighting = (0.5, 0.5, 0.)

produce_hk_stacking(hk_data, channel, weighting=weighting, save_file='{}.{}_{}_all.png'.format(target_station, channel, rf_type))
# produce_hk_stacking(hk_data, channel, weighting=(1, 0, 0))

In [ ]:
produce_hk_stacking(hk_bad, channel, weighting=weighting)

In [ ]:
produce_hk_stacking(hk_all, channel, weighting=weighting)

In [ ]:
def plot_polar_source_histogram(rf_stream, dist_range=(30, 90), radial_bins=3, polar_bins=24):

    from mpl_toolkits.axes_grid1 import make_axes_locatable

    azimut = np.deg2rad(np.array([tr.stats.back_azimuth for tr in rf_stream]))
    radius = np.array([tr.stats.distance for tr in rf_stream])

    # define binning
    rbins = np.linspace(dist_range[0], dist_range[1], radial_bins + 1)
    abins = np.linspace(0, 2*np.pi, polar_bins + 1)

    # calculate histogram
    hist, _, _ = np.histogram2d(azimut, radius, bins=(abins, rbins))
    A, R = np.meshgrid(abins, rbins)

    # plot
    fig = plt.figure(figsize=(10,10))
    ax = plt.subplot(111, projection="polar")
    pc = ax.pcolormesh(A, R, hist.T, cmap="magma_r", antialiased=True)
    ax.grid(linestyle=':')
    ax.set_yticks(rbins)
    ax.set_yticklabels(['{:2g}'.format(d) + '°' for d in rbins])
    ax.tick_params(labelsize=14)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(75)

#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("right", size="95%", pad=0.05)
#     cb = fig.colorbar(pc, cax=cax)
    cb = fig.colorbar(pc, fraction=0.042, pad=0.1)
    cb.set_label('Event count', fontsize=12)
    cb.ax.tick_params(labelsize=12)
    
    plt.plot(azimut, radius, 'o', color='#00c00080', markersize=10, fillstyle='none', markeredgewidth=2)
    ax.set_rorigin(0)
    
    return fig

In [ ]:
# Look at azimuth histogram
fig = plot_polar_source_histogram(rf_stream_A)
fig.gca().set_title("Polar distribution of source events for {}".format(target_station), fontsize=16)
plt.show()

### Look at just the rays from the north

In [ ]:
rf_stream_A_northerly = rf.RFStream([tr for tr in rf_stream_A if tr.stats.back_azimuth >= (360-45) or tr.stats.back_azimuth <= 45])
len(rf_stream_A_northerly)

In [ ]:
_ = rf_plot_utils.plot_rf_stack(rf_stream_A_northerly.sort(['distance']), trace_height=0.2, time_window=(-10, 30))

In [ ]:
hk_data_North = {channel: [tr for tr in rf_stream_A_northerly]}

In [ ]:
produce_hk_stacking(hk_data_North, channel, weighting=weighting, save_file='{}.{}_{}_north.png'.format(target_station, channel, rf_type))

### Look at just the rays from the east

In [ ]:
rf_stream_A_easterly = rf.RFStream([tr for tr in rf_stream_A if 45 <= tr.stats.back_azimuth <= (90+45)])
len(rf_stream_A_easterly)

In [ ]:
_ = rf_plot_utils.plot_rf_stack(rf_stream_A_easterly.sort(['distance']), trace_height=0.2, time_window=(-10, 30))

In [ ]:
hk_data_East = {channel: [tr for tr in rf_stream_A_easterly]}

In [ ]:
produce_hk_stacking(hk_data_East, channel, weighting=weighting, save_file='{}.{}_{}_east.png'.format(target_station, channel, rf_type))
# produce_hk_stacking(hk_data_East, channel, weighting=(1,0,0))
# produce_hk_stacking(hk_data_East, channel, weighting=(0,1,0))
# produce_hk_stacking(hk_data_East, channel, weighting=(0,0,1))

***

## Focus in on very narrow groups of events which we anticipate will sample same region of Moho lithography

In [ ]:
# First the strongest sector between 90 and 105 degrees of azimuth, distance up to 50 degrees
rf_stream_A_easterly2 = rf.RFStream([tr for tr in rf_stream_A if 90 <= tr.stats.back_azimuth <= 105 and tr.stats.distance <= 50])
len(rf_stream_A_easterly2)

In [ ]:
_ = rf_plot_utils.plot_rf_stack(rf_stream_A_easterly2.sort(['distance']), trace_height=0.2, time_window=(-10, 30))

In [ ]:
hk_data_East2 = {channel: [tr for tr in rf_stream_A_easterly2]}

In [ ]:
produce_hk_stacking(hk_data_East2, channel, weighting=weighting, save_file='{}.{}_{}_east2.png'.format(target_station, channel, rf_type))

In [ ]:
# Second strongest sector between 0 and 15 degrees of azimuth, distance from 50 up to 70 degrees
rf_stream_A_northerly2 = rf.RFStream([tr for tr in rf_stream_A if 0 <= tr.stats.back_azimuth <= 15 and 50 <= tr.stats.distance <= 70])
len(rf_stream_A_northerly2)

In [ ]:
_ = rf_plot_utils.plot_rf_stack(rf_stream_A_northerly2.sort(['distance']), trace_height=0.2)

In [ ]:
hk_data_North2 = {channel: [tr for tr in rf_stream_A_northerly2]}

In [ ]:
produce_hk_stacking(hk_data_North2, channel, weighting=weighting, save_file='{}.{}_{}_north2.png'.format(target_station, channel, rf_type))